# <font color='black'> Практическое занятие 3 </font>
## <font color='black'> Анализ панельных данных. Модели с фиксированными эффектами (FE-models) и модели со случайными эффектами (RE-models) </font>
На практическом занятии 3 мы рассмотрим модели со случайными эффектами, модели с фиксированными эффектами на временные периоды, а также модели, учитывающие разную взаимосвязь предиктора и зависимой переменной. 
Продолжим работать с той же базой данных:
* country -  страна: массив содержит данные по 27 посткоммунистическим странам
* period - период: данные усреднены по пятилеткам начиная с 1989 г. 
* fh_polity - индекс демократии, полученный на основе показателей Freedom House, Polity. Непрерывная шкала от 0 до 10, где 10 означает максимальное значение уровня демократии. 
* state_capacity - индекс государственной состоятельности. Приведен к непрерывной шкале от 0 до 10, где 10 - наиболее высокое значение государственной состоятельности. 

Подгрузим необходимые библиотеки и откроем массив "lab2.dta". 

In [ ]:
import pandas as pd
import statsmodels.formula.api as statf 
import numpy as np
import numpy.linalg as la
import seaborn as sb 
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from scipy import stats

In [ ]:
data = pd.read_stata("lab2.dta")
data.head(10)

Оценим уже знакомую Вам модель с фиксированными эффектами на страны с внутригрупповым преобразованием. 

In [ ]:
data1 = data.set_index(['country', 'period'])
model1 = PanelOLS.from_formula('fh_polity ~ state_capacity + EntityEffects', data = data1).fit()
print(model1)

Сравним полученные оценки с результатами оценивания модели со случайными эффектами. 
* Вспомните допущения данной модели и оцените критически 
* Какой метод оценивания используется в данном случае?

In [ ]:
model2 = RandomEffects.from_formula('fh_polity ~ 1 + state_capacity', data = data1).fit()
print(model2)

Применим тест Хаусмана для проверки гипотезы об отсутствии корреляции между индивидуальными эффектами и предикторами. В Python отсутствует автоматическая функция реализации данного теста, поэтому зададим в явном виде статистику для проверки гипотезы и ее распределение. Проинтерпретируйте результаты тестирования.

In [ ]:
b_fe = model1.params
b_re = model2.params
b_re_new = np.asarray(b_re)
b_re_new = np.delete(b_re_new, 0) 

cov_fe = model1.cov
cov_re = model2.cov
cov_re = np.asarray(model2.cov)
cov_re
cov_re_new = np.delete(np.delete(cov_re, 0, axis = 0), 0, 1)
hausman = (b_fe - b_re_new).dot(la.inv(cov_fe - cov_re_new).dot(b_fe - b_re_new))
df = b_fe.size
pvalue = stats.chi2.sf(hausman, df)
print('Chi-squared:', hausman, '\n' 'P-value:', pvalue, '\n' 'Degrees of freedom:', df)

Следующая модель учитывает фиксированные эффекты на временные периоды. 
* Проинтерпретируйте хотя бы один коэффициент при временном периоде
* Что теперь отражает оценка коэффициента при state_capacity? 

In [ ]:
model3 = statf.ols('fh_polity ~ state_capacity + C(period)', data).fit()
print(model3.summary())

Теперь оценим наименее экономную модель, с включением эффектов и на страны, и на временные периоды. Используем внутригрупповое преобразование. 
* Каковы ограничения указанной модели? 
* Что отражает оценка коэффициента при state_capacity? 

In [ ]:
model4 = PanelOLS.from_formula('fh_polity ~ state_capacity + EntityEffects + TimeEffects', data1).fit()
print(model4)

Проверим, можем ли мы ограничиться только эффектами на пространственные единицы. Для этого сравним две вложенные модели: только с пространственными эффектами, и twoway model.
* Сделайте вывод на основе p-value. 
* Сопоставьте со своими содержательными соображениями, какая модель более предпочтительна. 

In [ ]:
model5 = statf.ols('fh_polity ~ state_capacity + C(country)', data).fit()
model6 = statf.ols('fh_polity ~ state_capacity + C(country) + C(period)', data).fit()
F = ((model6.rsquared - model5.rsquared)/(model6.params.size - model5.params.size))/((1-model6.rsquared)/(model6.nobs - model6.params.size))
F
pvalue = stats.f.sf(F, model6.params.size - model5.params.size, model6.nobs - model6.params.size)
print('F-statistic:', F, '\n' 'P-value:', pvalue, '\n' 'df1:', model6.params.size - model5.params.size, '\n' 'df2:', model6.nobs - model6.params.size )

Проиллюстрируем, как можно учесть разницу в характере взаимосвязи гос. состоятельности и полит. режима. Проинтерпретируйте
* оценку константы 
* оценку коэффициента при одной из дамми-переменных
* оценку коэффициента при state_capacity
* оценку коэффициента хотя бы при одной переменной взаимодействия дамми на пространственную единицу и state_capacity

In [ ]:
model7 = statf.ols('fh_polity ~ C(country)*state_capacity', data).fit()
print(model7.summary())